In [1]:
from importlib import reload
import sys
from imp import reload
import warnings
import pandas as pd
import re
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
import nltk
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Dense , Input , CuDNNLSTM , Embedding, Dropout , Activation, GRU, Flatten
from keras.layers import Bidirectional, GlobalMaxPool1D, concatenate, dot
from keras.models import Model, Sequential
from keras.layers import LSTM
from phased_lstm_keras.PhasedLSTM import PhasedLSTM as PLSTM
import tensorflow as tf

warnings.filterwarnings('ignore')
if sys.version[0] == '2':
    reload(sys)
    sys.setdefaultencoding("utf-8")

Using TensorFlow backend.
c:\users\user\anaconda3\envs\cuda\lib\site-packages\tensorflow\python\framework\dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
c:\users\user\anaconda3\envs\cuda\lib\site-packages\tensorflow\python\framework\dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
c:\users\user\anaconda3\envs\cuda\lib\site-packages\tensorflow\python\framework\dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
c:\users\user\anaconda3\envs\cuda\lib\site-packages\t

In [2]:
df = pd.read_csv('questions.csv')
df = df.drop(['id','qid1','qid2'],axis=1)
df.head()

,question1,question2,is_duplicate
0,What is the step by step guide to invest in sh...,What is the step by step guide to invest in sh...,0
1,What is the story of Kohinoor (Koh-i-Noor) Dia...,What would happen if the Indian government sto...,0
2,How can I increase the speed of my internet co...,How can Internet speed be increased by hacking...,0
3,Why am I mentally very lonely? How can I solve...,Find the remainder when [math]23^{24}[/math] i...,0
4,"Which one dissolve in water quikly sugar, salt...",Which fish would survive in salt water?,0


In [3]:

#nltk.download('stopwords')


stop_words = set(stopwords.words("english")) 
lemmatizer = WordNetLemmatizer()

def clean_text(text):
    text = re.sub(r'[^\w\s]','',text, re.UNICODE)
    text = text.lower()
    text = [lemmatizer.lemmatize(token) for token in text.split(" ")]
    text = [lemmatizer.lemmatize(token, "v") for token in text]
#     text = [word for word in text if not word in stop_words]
    text = " ".join(text)
    return text

In [4]:
#nltk.download('wordnet')


# 이 곳 때문에 tokenizer가 가능 
df['question1'] = df.question1.apply(lambda x: clean_text(str(x)))
df.head()

,question1,question2,is_duplicate
0,what be the step by step guide to invest in sh...,What is the step by step guide to invest in sh...,0
1,what be the story of kohinoor kohinoor diamond,What would happen if the Indian government sto...,0
2,how can i increase the speed of my internet co...,How can Internet speed be increased by hacking...,0
3,why be i mentally very lonely how can i solve it,Find the remainder when [math]23^{24}[/math] i...,0
4,which one dissolve in water quikly sugar salt ...,Which fish would survive in salt water?,0


In [5]:
df['question2'] = df.question2.apply(lambda x: clean_text(str(x)))
df.head()

,question1,question2,is_duplicate
0,what be the step by step guide to invest in sh...,what be the step by step guide to invest in sh...,0
1,what be the story of kohinoor kohinoor diamond,what would happen if the indian government ste...,0
2,how can i increase the speed of my internet co...,how can internet speed be increase by hack thr...,0
3,why be i mentally very lonely how can i solve it,find the remainder when math2324math be divide...,0
4,which one dissolve in water quikly sugar salt ...,which fish would survive in salt water,0


In [6]:
# 위애서 lambda 로 전처리 해줘서 가능

total_text = pd.concat([df['question1'], df['question2']]).reset_index(drop=True)
max_features = 200
tokenizer = Tokenizer(num_words=max_features)
tokenizer.fit_on_texts(total_text)
question_1_sequenced = tokenizer.texts_to_sequences(df['question1'])
question_2_sequenced = tokenizer.texts_to_sequences(df['question2'])

In [7]:
maxlen = 100
question_1_padded = pad_sequences(question_1_sequenced, maxlen=maxlen)
question_2_padded = pad_sequences(question_2_sequenced, maxlen=maxlen)

In [8]:
y = df['is_duplicate']

In [9]:
embedding_size = 32

inp1 = Input(shape=(100,))
inp2 = Input(shape=(100,))

x1 = Embedding(max_features, embedding_size)(inp1)
x2 = Embedding(max_features, embedding_size)(inp2)

x3 = Bidirectional(CuDNNLSTM(16, return_sequences = True))(x1)
x4 = Bidirectional(CuDNNLSTM(16, return_sequences = True))(x2)

x5 = GlobalMaxPool1D()(x3)
x6 = GlobalMaxPool1D()(x4)

x7 =  dot([x5, x6], axes=1)

x8 = Dense(10, activation='relu')(x7)
x9 = Dropout(0.05)(x8)
output = Dense(1, activation="sigmoid")(x9)

model = Model(inputs=[inp1, inp2], outputs=output)
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
batch_size = 100
epochs = 3
model.fit([question_1_padded, question_2_padded], y, batch_size=batch_size, epochs=epochs, validation_split=0.2)

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.
Instructions for updating:
Deprecated in favor of operator or tf.math.divide.
Train on 323480 samples, validate on 80871 samples
Epoch 1/3
323480/323480 [==============================] - 90s 279us/step - loss: 0.5852 - accuracy: 0.6957 - val_loss: 0.5476 - val_accuracy: 0.7239
Epoch 2/3
323480/323480 [==============================] - 82s 254us/step - loss: 0.5419 - accuracy: 0.7241 - val_loss: 0.5257 - val_accuracy: 0.7351
Epoch 3/3
323480/323480 [==============================] - 82s 254us/step - loss: 0.5254 - accuracy: 0.7348 - val_loss: 0.5233 - val_accuracy: 0.7320


In [10]:
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 100)          0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            (None, 100)          0                                            
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, 100, 32)      6400        input_1[0][0]                    
__________________________________________________________________________________________________
embedding_2 (Embedding)         (None, 100, 32)      6400        input_2[0][0]                    
____________________________________________________________________________________________

In [11]:
tf.__version__

from keras import backend as K
K.tensorflow_backend._get_available_gpus()


['/job:localhost/replica:0/task:0/device:GPU:0']